# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [1]:
import pandas as pd
import numpy as np

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [2]:
df = pd.read_csv('fires-all.csv')
print(df.describe())

                 id    superficie           lat           lng  \
count  8.264000e+04  82640.000000  82616.000000  82616.000000   
mean   2.007259e+09     19.888085     41.763721     -5.664360   
std    4.247106e+06    223.787536      2.019672      4.394031   
min    2.001010e+09      1.000000      0.490720  -1000.000000   
25%    2.003390e+09      1.500000     40.876544     -7.257298   
50%    2.006330e+09      3.000000     42.371123     -6.019478   
75%    2.011150e+09      7.150000     43.143145     -4.303078   
max    2.015510e+09  28879.100000     87.824157    242.755603   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82640.000000  82640.000000  82640.000000  82640.000000   
mean          0.741626      7.850278     28.411375     77.795946   
std           0.437743      5.419922     11.727155     98.867769   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

## Preprocessing

### Tractament de missing values

In [3]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

['lat', 'lng', 'causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar lat i long

Per arreglar lat i long ho fem en un altre document "corregirCoordenades.py", ja que és un procés més lent.
Es dedica a comprovar les coordenades de cada instància per veure si es corresponen a un diccionari generat
anteriorment amb el fitxer "obtenirCoordenades.py". Hi ha una explicació més extensa a la documentació.

In [4]:
df = pd.read_csv('coordsCorregides.csv')
# Comprovem que ja no queden nulls a coordenades
print(df.columns[df.isnull().any()].tolist())

['causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar muertos, heridos y causa_supuesta

In [5]:
# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)
# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que és o 1 o nan per tant els posem a 0
df['causa_supuesta'].fillna(0, inplace=True)

#### Arreglar gastos y perdidas

Per a fer aquesta part hem de netejar una mica més el dataset, la columna de "idmunicipio" és incorrecte, ja que
diferents municipis comparteixen el mateix id tot i està a cada punta del territori. També eliminem la columna municipio
ja que amb la latitud i longitud ja sabem on està localitzat l'incendi.

Primer separem target del dataset, volem predir la superfície cremada donat un incendi per
tant separem aquesta variable de la resta.

https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e

In [6]:
# Arreglar valors massa petits posant-lo a NaN
df.at[df.loc[df["gastos"] < 200].index, 'gastos'] = np.NaN
df.at[df.loc[df["perdidas"] < 200].index, 'perdidas'] = np.NaN
df["fecha"] = df["fecha"].apply(lambda f: f.replace('-', ''))
df.drop(['idmunicipio', 'municipio'], axis=1, inplace=True)
print(df.head())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Separem el target
# X = df.drop(['superficie', 'idmunicipio', 'municipio'], axis=1)
# y = df['superficie']
#


# # Apliquem KNN per tal d'emplenar els missing values
from sklearn.impute import KNNImputer
imputer = KNNImputer()
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Fem knn
# df['gastos'].fillna(0, inplace=True)
# df['perdidas'].fillna(0, inplace=True)

# df.describe()

print(df.columns[df.isnull().any()].tolist())

           id  superficie     fecha        lat       lng  latlng_explicit  \
0  2001010001        3.70  20010318  42.954656 -2.325719                1   
1  2001010004        1.50  20010324  42.552183 -2.640673                1   
2  2001010005        1.50  20010416  42.493480 -2.575597                0   
3  2001010008        7.50  20010525  42.946479 -2.485163                1   
4  2001010017        1.04  20010720  43.091427 -3.069773                0   

   idcomunidad  idprovincia  causa  causa_supuesta  causa_desc  muertos  \
0            1            1      4             0.0           2      0.0   
1            1            1      2             1.0          10      0.0   
2            1            1      2             0.0          10      0.0   
3            1            1      4             0.0           2      0.0   
4            1            1      5             0.0           0      0.0   

   heridos  time_ctrl  time_ext  personal  medios  gastos  perdidas  
0      0.0      

In [9]:
print(df.head())
print(df.describe())

df.to_csv("noNaNOriginalLatLng.csv")



             id  superficie     fecha       lat       lng  latlng_explicit  \
0  0.000000e+00    0.000093  0.001538  0.949757  0.707099              1.0   
1  2.068965e-07    0.000017  0.001580  0.924687  0.692873              1.0   
2  2.758621e-07    0.000017  0.002232  0.921030  0.695812              0.0   
3  4.827586e-07    0.000225  0.003004  0.949247  0.699897              1.0   
4  1.103448e-06    0.000001  0.004386  0.958276  0.673491              0.0   

   idcomunidad  idprovincia  causa  causa_supuesta  causa_desc  muertos  \
0          0.0          0.0    0.6             0.0    0.020202      0.0   
1          0.0          0.0    0.2             1.0    0.101010      0.0   
2          0.0          0.0    0.2             0.0    0.101010      0.0   
3          0.0          0.0    0.6             0.0    0.020202      0.0   
4          0.0          0.0    0.8             0.0    0.000000      0.0   

   heridos  time_ctrl  time_ext  personal    medios    gastos  perdidas  
0     